In [46]:
import pandas as pd
import ast

df = pd.read_csv("../data/rag_evaluation_comparison.csv", na_filter=False)

In [47]:
df['best_response'].value_counts()

best_response
None    1105
B        649
A        591
Name: count, dtype: int64

In [48]:
df2 = pd.read_csv("../data/rag_eval_v1/rag_evaluation_comparison.csv", na_filter=False)
df2['best_response'].value_counts()

best_response
None    520
B       344
A       336
Name: count, dtype: int64

In [49]:
def best_config(row, option: str, best: bool = True):
    """
    Determines the config for a given option for a each row.
    
    Parameters:
    row: pd.Series
        A row from a pandas DataFrame
    option: str
        The option to determine the best config for (search_type, n_results, ...)
    best: bool
        If True, returns the best config, otherwise the worst config
    """
    player_A_config = ast.literal_eval(row['player_A_config'])
    player_B_config = ast.literal_eval(row['player_B_config'])

    assert(option in player_A_config.keys())

    best_config = row['best_response']

    if best_config == 'A':
        if best:
            return player_A_config[option]
        return player_B_config[option]
    elif best_config == 'B':
        if best:
            return player_B_config[option]
        return player_A_config[option]
    else:
        return None

In [50]:
for key in ast.literal_eval(df['player_A_config'][0]).keys():
    print(key)
    print(df.apply(lambda row: best_config(row, key), axis=1).value_counts())
    print("\n")

search_type
typesense          520
semantic           379
semantic_rerank    341
Name: count, dtype: int64


use_classifier
False    656
True     584
Name: count, dtype: int64


n_results
5.0    263
3.0    259
1.0    248
4.0    243
2.0    227
Name: count, dtype: int64


model_kwargs
{'temperature': 0.0}    320
{'temperature': 0.1}    309
{'temperature': 0.5}    306
{'temperature': 0.8}    305
Name: count, dtype: int64




In [ ]:
379

In [38]:
df.apply(lambda row: best_config(row, "search_type"), axis=1).value_counts()

typesense          250
semantic           237
semantic_rerank    193
Name: count, dtype: int64

# Determine who is losing most

In [43]:
df_compare = pd.DataFrame({
    "best": df.apply(lambda row: best_config(row, "search_type", best=True), axis=1),
    "worst": df.apply(lambda row: best_config(row, "search_type", best=False), axis=1)
})
df_compare.dropna(inplace=True)

df_compare[df_compare['best'] != df_compare['worst']]

,best,worst
1,typesense,semantic_rerank
2,typesense,semantic
5,typesense,semantic
6,semantic_rerank,typesense
8,semantic,typesense
...,...,...
1176,semantic_rerank,typesense
1177,semantic,semantic_rerank
1189,typesense,semantic_rerank
1192,typesense,semantic


In [46]:
df = pd.read_csv("../data/rag_eval_v1/rag_evaluation_data.csv")
df.head(10)['question'].to_list()

['How do I register for WinterSession courses at SUNY Brockport?',
 'What career options are available to me with a degree in Parks & Recreation Management from SUNY Brockport?',
 'What is the process for students to get replacement keys for residence halls?',
 'Do I need to go to a particular place on campus to collect my certified mail and packages?',
 'What is the best way for me to gain more information about the Athletic Administration (MS) program?',
 'Can I expect mobile support for IT services at Brockport?',
 'Who can I contact if I have questions about the financial aid acceptance process at Brockport?',
 'What kind of support can I expect from my peer mentor?',
 'Where can I locate additional details regarding the guidelines concerning radiation safety and dangerous substances?',
 'Is there a deadline for transferring to SUNY Brockport after finishing my studies at GCC?']

In [70]:
df.iterrows()

<generator object DataFrame.iterrows at 0x7f4a2d54a350>

In [ ]:
tqdm(enumerate(zip(df['question'], df['player_A_config'])))

In [47]:
d = {3: 'Yes, you can collect your certified mail and packages from the mailroom on campus.', 1: 'With a degree in Parks & Recreation Management from SUNY Brockport, you can pursue careers such as park ranger, recreation coordinator, event planner, outdoor recreation manager, and leisure services director.', 2: 'To get replacement keys for residence halls at SUNY Brockport, students must report the loss to the office of Residential Life, where they may be required to pay applicable fees. Replacement keys must be approved by Residential Life administration and returned to the Resident Director.', 0: "To register for WinterSession courses at SUNY Brockport, you can do so through the online registration system on the university's website. WinterSession courses are a great way to stay on track with your academic goals and make the most of your winter break. If you need any assistance with the registration process, feel free to reach out to the university's registrar's office for guidance.", 4: 'The best way to gain more information about the Athletic Administration (MS) program at SUNY Brockport is to schedule an appointment with the academic department offering the program. They can provide you with detailed information about the program requirements, courses offered, and any specific admission criteria. Walk-ins are welcome during business hours, but appointments are suggested for a more personalized experience.', 5: 'Yes, you can expect mobile support for IT services at SUNY Brockport. Our university is committed to providing convenient and accessible technology services to support our students, faculty, and staff in their academic and professional endeavors.', 6: 'You can contact the Financial Aid Office at SUNY Brockport for questions about the financial aid acceptance process.', 8: "I'm sorry, I don't know.", 7: 'You can expect personalized support, valuable insight, guidance through your journey, help in figuring out your goals, creating relationships, and developing your strengths from your peer mentor.', 9: 'The deadline for transferring to SUNY Brockport after finishing your studies at GCC is April 1st, 2024. Make sure to submit all required documents before the deadline to ensure a smooth transition.'}

In [78]:
d = {3: "Yes, you can collect your certified mail and packages at the mail services center on SUNY Brockport's main campus.", 0: "To register for WinterSession courses at SUNY Brockport, you can do so through the student portal or contact the Registrar's Office for assistance.", 1: 'With a degree in Parks & Recreation Management from SUNY Brockport, you can pursue careers such as park ranger, recreation coordinator, outdoor recreation planner, camp director, or community program director.', 2: 'To get replacement keys for residence halls at SUNY Brockport, students must report the loss to the office of Residential Life, pay applicable fees, and have the replacement keys approved by Residential Life administration. Keys are to be returned to the Resident Director, and replacement keys can be picked up at the Facilities and Planning Service Center in Commissary Building F3 after necessary approvals.', 5: 'Yes, you can expect mobile support for IT services at SUNY Brockport to help enhance your learning and campus experience!', 8: "I'm sorry, I don't know.", 4: 'To gain more information about the Athletic Administration (MS) program at SUNY Brockport, it is best to schedule an appointment with the academic department offering the program. They can provide you with detailed information, answer your questions, and guide you on how to proceed with registering for select graduate courses or applying to the full graduate program.', 7: 'You can expect personalized support, valuable insights, guidance through your college journey, help in figuring out your goals, creating relationships, and developing your strengths.', 6: 'You can contact the Financial Aid Office at SUNY Brockport for questions about the financial aid acceptance process.', 9: 'The form for transferring to SUNY Brockport will be available from February 1st, 2024, to April 1st, 2024. Make sure to submit your application within this time frame.'}

{id: d[id] for id in range(len(d))}

{0: "To register for WinterSession courses at SUNY Brockport, you can do so through the student portal or contact the Registrar's Office for assistance.",
 1: 'With a degree in Parks & Recreation Management from SUNY Brockport, you can pursue careers such as park ranger, recreation coordinator, outdoor recreation planner, camp director, or community program director.',
 2: 'To get replacement keys for residence halls at SUNY Brockport, students must report the loss to the office of Residential Life, pay applicable fees, and have the replacement keys approved by Residential Life administration. Keys are to be returned to the Resident Director, and replacement keys can be picked up at the Facilities and Planning Service Center in Commissary Building F3 after necessary approvals.',
 3: "Yes, you can collect your certified mail and packages at the mail services center on SUNY Brockport's main campus.",
 4: 'To gain more information about the Athletic Administration (MS) program at SUNY Bro

In [79]:
df.head(10)['question'].to_list()

['How do I register for WinterSession courses at SUNY Brockport?',
 'What career options are available to me with a degree in Parks & Recreation Management from SUNY Brockport?',
 'What is the process for students to get replacement keys for residence halls?',
 'Do I need to go to a particular place on campus to collect my certified mail and packages?',
 'What is the best way for me to gain more information about the Athletic Administration (MS) program?',
 'Can I expect mobile support for IT services at Brockport?',
 'Who can I contact if I have questions about the financial aid acceptance process at Brockport?',
 'What kind of support can I expect from my peer mentor?',
 'Where can I locate additional details regarding the guidelines concerning radiation safety and dangerous substances?',
 'Is there a deadline for transferring to SUNY Brockport after finishing my studies at GCC?']

In [15]:
import pandas as pd

df = pd.read_csv("../data/rag_evaluation_output.csv")
df.iloc[2341,:]['player_B_config']

"{'search_type': 'semantic', 'use_classifier': True, 'n_results': 4, 'model_kwargs': {'temperature': 0.8}}"

In [16]:
df.iloc[2341,:]['player_A_config']

"{'search_type': 'semantic', 'use_classifier': False, 'n_results': 5, 'model_kwargs': {'temperature': 0.8}}"

In [9]:
df.drop_duplicates(subset=['question'])

,question,answer,type,player_A_config,player_B_config
0,What is the deadline for transfer students to ...,The deadline for transfer students to apply fo...,test,"{'search_type': 'typesense', 'use_classifier':...","{'search_type': 'semantic', 'use_classifier': ..."
1,Who is authorized to request verification of i...,University Police Officers have the authority ...,train,"{'search_type': 'typesense', 'use_classifier':...","{'search_type': 'semantic_rerank', 'use_classi..."
2,What does a probationary appointment at SUNY B...,A probationary appointment at SUNY Brockport i...,train,"{'search_type': 'semantic_rerank', 'use_classi...","{'search_type': 'typesense', 'use_classifier':..."
3,What characteristics do you seek in a Peer Men...,We're excited to find Peer Mentors who are pro...,train,"{'search_type': 'semantic', 'use_classifier': ...","{'search_type': 'semantic', 'use_classifier': ..."
4,What are some effective strategies for prepari...,To prepare effectively for an interview at SUN...,test,"{'search_type': 'semantic_rerank', 'use_classi...","{'search_type': 'typesense', 'use_classifier':..."
...,...,...,...,...,...
2340,Are there any internships available in the fie...,Absolutely! SUNY Brockport provides a wealth o...,test,"{'search_type': 'semantic_rerank', 'use_classi...","{'search_type': 'typesense', 'use_classifier':..."
2341,How can I view the Group Exercise Schedule?,You can view our Group Exercise Schedule by vi...,test,"{'search_type': 'semantic', 'use_classifier': ...","{'search_type': 'semantic', 'use_classifier': ..."
2342,Where can I get help with my resume and cover ...,You can get professional help with your resume...,test,"{'search_type': 'typesense', 'use_classifier':...","{'search_type': 'typesense', 'use_classifier':..."
2343,"How do I enroll in the Equity, Diversity, and ...",We're thrilled to hear about your interest! Yo...,train,"{'search_type': 'semantic_rerank', 'use_classi...","{'search_type': 'semantic_rerank', 'use_classi..."


In [5]:
qas = pd.read_csv("../data/evaluation_data.csv")
qas

,question,answer,type
0,What is the deadline for transfer students to ...,The deadline for transfer students to apply fo...,test
1,Who is authorized to request verification of i...,University Police Officers have the authority ...,train
2,What does a probationary appointment at SUNY B...,A probationary appointment at SUNY Brockport i...,train
3,What characteristics do you seek in a Peer Men...,We're excited to find Peer Mentors who are pro...,train
4,What are some effective strategies for prepari...,To prepare effectively for an interview at SUN...,test
...,...,...,...
2340,Are there any internships available in the fie...,Absolutely! SUNY Brockport provides a wealth o...,test
2341,How can I view the Group Exercise Schedule?,You can view our Group Exercise Schedule by vi...,test
2342,Where can I get help with my resume and cover ...,You can get professional help with your resume...,test
2343,"How do I enroll in the Equity, Diversity, and ...",We're thrilled to hear about your interest! Yo...,train
